In [19]:
import cv2
import skvideo
import math
import os
import imageio
import numpy as np
cv2.__version__

'3.4.1'

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
from skimage import filters

class Video_Preprocessing(object):
    """
    convert video file  to the image's folders by period , size  and max_length
    if video seperated by a black frame each video will be write in its own folder 
    video_file is the name of video file
    folder_name - main folder name for the file
    period - step with the frame turns to the image
    max_length - max length in frames for the folder of images
    size - if need to resize the image
    video_count - first number of the image's folder
    cut - a flag to cut the original image into x1, x2, y1, y2 image
    mode = {'gray', 'segment', 'lines', 'edges', 'diff', 'orb',  'filter'} is the way to preparation of images
    verbose is the flag of debarging
    
    """
    def __init__(self, 
                 mode='filter',  # method applied to frame
                 max_length=np.inf, # max length of image's folder
                 period=1, # step of frame to the image
                 size=(224,224), # resize the initial image
                 cut=True, # cuttinf the initial image
                 cutting=(400, 1000, 120, 1000), # points of the cutting image
                 verbose=False # flag deburging
                ):
        self.mode = mode
        self.max_length = max_length
        self.period = period
        self.size = size
        self.cut = cut
        self.cutting = cutting
        self.verbose = verbose
        
    def check_make_dir(self, folder):
        """
        check the existing of the folder
        if not make it
        """
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    def video_to_image_folders(self,
                              video_file, # file of video 
                              folder_name, # the main folder that containe daughter's folders for each video
                              video_count # first number of daughter'sfolder 
                             ):
        """
        convert video file  to the image's folders by period , size  and max_length
        if video seperated by a black frame each video will be write in its own folder 
        video_file is the name of video file
        folder_name - main folder name for the file
        """
        image_count = 0 # first image
        
        video = cv2.VideoCapture(video_file)
        while video.isOpened():
            success, image = video.read()
            if not success:
                print 'No read may be incorect the video file or the name, or the end of the video'
                break
            
            if np.any(image == None):
                continue
            # simply cut the most important image segment
            if self.cut:
                x1, x2, y1, y2 = self.cutting
                image = image[x1:x2, y1:y2]
                
        
            # check the black seperating 
            black = np.all(image <= 0.01)
            # seperate by black   
            if black:
                # if there is a black image turn to the next folder 
                if image_count != 0:
                    video_count += 1
                image_count = 0
                continue
                
            else:
                # if not seperating creat the file to write in the same folder
                if not os.path.exists(folder_name +'/video_' + str(video_count).zfill(5)):
                    os.makedirs(folder_name +'/video_' + str(video_count).zfill(5))
                                
                # if yes seperating creat the new folder
                else:
                    # check the length of the folder 
                    if len(os.listdir(folder_name +'/video_' + str(video_count).zfill(5))) >= self.max_length:
                        video_count += 1
                        os.makedirs(folder_name +'/video_' + str(video_count).zfill(5))
       
                # period chosen the image
                if not (image_count% self.period == 0): # use only images with the period
                    continue
                
                filename = (folder_name +'/video_' +str(video_count).zfill(5) + '/' 
                                  + str(image_count+1).zfill(5) +'_image.jpg' )              
                
                # HSV represantation of the imade
                image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                
                # gray presantation of image
                if self.mode =='gray':
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                            
                # find lines on the image and add lines to the image
                elif self.mode =='lines':
                    edges = cv2.Canny(image, 4, 70, apertureSize=3)
                    lines = cv2.HoughLines(edges, 1, np.pi/180, 200, 100, 10)
                    
                    for xx in lines:
                        rho, theta = xx[0]
                        a = np.cos(theta)
                        b = np.sin(theta)
                        x0 = a * rho
                        y0 = b * rho
                        x1 = int(x0 + 1000*(-b))
                        y1 = int(y0 + 1000* (a))
                        x2 = int(x0 - 1000*(-b))
                        y2 = int(y0 - 1000* (a))
                            
                        # use lines have only the angle
                        if (theta > 0.7) & (theta < 1.8):
                            image = cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                               
                # find contours on the image and turn the image in this style
                elif self.mode =='edges':
                    image = cv2.Canny(image, 60, 100, apertureSize=3)
                                        
                # find thresholds on the image and turn the image in this style    
                elif self.mode =='thresh':
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    image = cv2.medianBlur(image, 5)
                    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                                      cv2.THRESH_BINARY, 13, 4)
                
                # find background on the video and diff the backgrounf from the image  
                elif self.mode =='segment':
                    fgbg = cv2.createBackgroundSubtractorMOG2()
                    image = fgbg.apply(image)
                        
                # find the difference between two serial images
                elif self.mode =='diff':
                    if image_count == 0:
                        image_old = image.copy()
                    image_diff = cv2.bitwise_xor(image, image_old)
                    image_old = image.copy()
                    image = cv2.cvtColor(image_diff, cv2.COLOR_BGR2GRAY)
                     
                # find the org poins in images
                elif self.mode =='orb':
                    # Initiate STAR detector
                    orb = cv2.ORB_create(scoreType=cv2.ORB_FAST_SCORE)

                    # find the keypoints with ORB
                    kp = orb.detect(image,None)

                    # compute the descriptors with ORB
                    kp, des = orb.compute(image, kp)

                    # draw only keypoints location,not size and orientation
                    image = cv2.drawKeypoints(np.zeros_like(image),kp, None, color=(255,255,255), flags=cv2.DrawMatchesFlags_DEFAULT)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                        
                # apply the filter to the images
                elif self.mode =='filter':
                    #filter parameters
                    blue_low = np.array([20, 30, 100])
                    blue_high = np.array([160, 250, 255])
                    # Threshold the HSV image to get only blue colors
                    mask = cv2.inRange(image, blue_low, blue_high)

                    # Bitwise-AND mask and original image
                    image = cv2.bitwise_and(image,image, mask= mask)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                        
                #resize image
                if self.size is not None:
                    image = cv2.resize(image, self.size, interpolation=cv2.INTER_NEAREST)
                                
                if self.verbose:
                    print filename, '  ',image.shape
                
                # write the image in file   
                try:
                    imageio.imwrite(filename, image)
                except ValueError:
                    continue

                image_count +=1
    
        numbers_of_video = np.zeros(video_count, dtype=np.int32)
    
        # count the number of images that wrote in folders as a file
        for i,f in enumerate(os.listdir(folder_name)):
            numbers_of_video[i] = len(os.listdir(folder_name +'/' + f))
   
        print '*** Done *** write in ', folder_name
        print 'model of preprocession is, ', self.mode
        print 'number of videos ', len(numbers_of_video)
        print 'max_image_len ', int(numbers_of_video.max())
        print 'min_image_len ', int(numbers_of_video.min())
        
    def video_folder_to_image_folders(self,
                                       video_folder, # folder of video or file
                                       folder_name=None, # name of main image' folder
                                      ):
        """
        convert a video folder with the same class of video to image's folders
        each video will be convert to its own daughter's image's folder
        video_folder - the folder contains the video files the same class
        folder_name - name of image's folders as rule the name of the class
        """
        if folder_name is None:
            if os.path.isdir(video_folder):
                folder_name = video_folder + '_images'
            elif os.path.isfile(video_folder):
                folder_name = video_folder.split('.')[0] + '_images'
        
    
        if os.path.isfile(video_folder):
            video_list = [os.path.join(os.getcwd(), video_folder)]
        elif os.path.isdir(video_folder):
            video_list = map(lambda x: os.path.join(os.getcwd(), video_folder, x), os.listdir(video_folder))
        else:
            print video_folder, 'not folder nor file'
            return
    
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
            n = 1
        else:
            n = len(os.listdir(folder_name)) + 1
    
        for video_file in video_list:
            self.video_to_image_folders(video_file, folder_name, n)
            n = len(os.listdir(folder_name)) + 1
    
    def different_video_folders_to_image_folders(self,
                        video_folders, # folder contain the video of different classes
                        folders_names=None # the list of clesses names
                        ):
        """
        convert folders of video with different classes to foldes of images
        video_folders - the folders contain the video folders or files of different classes
        folder_names - names of image's folders as rule the names of classes, the same length as the video folders
        """
        if not os.path.isdir(video_folders):
            print video_folders, ' is not a video folders'
            print 'get the video folders'
            return
        else:
            video_folders_list = sorted(map(lambda x: os.path.join(os.getcwd(), video_folders, x), 
                                     os.listdir(video_folders)))
        
        _folders_names = []
        for i, video_folder in enumerate(video_folders_list):
            if not folders_names:
                folder_name = video_folders + '_images/' +'class_'+str(i)
            else:
                assert len(folders_names) == len(video_folders_list)
                folder_name = folders_names[i]
        
            self.video_folder_to_image_folders(video_folder, folder_name)
            folder_name = os.path.join(ob.getcwd(), folder_name)
            _folders_names.append(folder_name)
            
        self.foldes = _folders_names
        print 'write the images in the ',_folders_names

In [22]:
vv = Video_Preprocessing(size=(224, 112), verbose=True)
print vv.size
vv.mode

(224, 112)


'filter'

In [54]:
#vv.different_video_folders_to_image_folders('video')

video_imades/class_0/video_00001/00001_image.jpg    (112, 224)
video_imades/class_0/video_00001/00002_image.jpg    (112, 224)
video_imades/class_0/video_00001/00003_image.jpg    (112, 224)
video_imades/class_0/video_00001/00004_image.jpg    (112, 224)
video_imades/class_0/video_00001/00005_image.jpg    (112, 224)
video_imades/class_0/video_00001/00006_image.jpg    (112, 224)
video_imades/class_0/video_00001/00007_image.jpg    (112, 224)
video_imades/class_0/video_00001/00008_image.jpg    (112, 224)
video_imades/class_0/video_00001/00009_image.jpg    (112, 224)
video_imades/class_0/video_00001/00010_image.jpg    (112, 224)
video_imades/class_0/video_00001/00011_image.jpg    (112, 224)
video_imades/class_0/video_00001/00012_image.jpg    (112, 224)
video_imades/class_0/video_00001/00013_image.jpg    (112, 224)
video_imades/class_0/video_00001/00014_image.jpg    (112, 224)
video_imades/class_0/video_00001/00015_image.jpg    (112, 224)
video_imades/class_0/video_00001/00016_image.jpg    (11

video_imades/class_0/video_00001/00132_image.jpg    (112, 224)
video_imades/class_0/video_00001/00133_image.jpg    (112, 224)
video_imades/class_0/video_00001/00134_image.jpg    (112, 224)
video_imades/class_0/video_00001/00135_image.jpg    (112, 224)
video_imades/class_0/video_00001/00136_image.jpg    (112, 224)
video_imades/class_0/video_00001/00137_image.jpg    (112, 224)
video_imades/class_0/video_00001/00138_image.jpg    (112, 224)
video_imades/class_0/video_00001/00139_image.jpg    (112, 224)
video_imades/class_0/video_00001/00140_image.jpg    (112, 224)
video_imades/class_0/video_00001/00141_image.jpg    (112, 224)
video_imades/class_0/video_00001/00142_image.jpg    (112, 224)
video_imades/class_0/video_00001/00143_image.jpg    (112, 224)
video_imades/class_0/video_00001/00144_image.jpg    (112, 224)
video_imades/class_0/video_00001/00145_image.jpg    (112, 224)
video_imades/class_0/video_00001/00146_image.jpg    (112, 224)
video_imades/class_0/video_00001/00147_image.jpg    (11

video_imades/class_0/video_00002/00106_image.jpg    (112, 224)
video_imades/class_0/video_00002/00107_image.jpg    (112, 224)
video_imades/class_0/video_00002/00108_image.jpg    (112, 224)
video_imades/class_0/video_00002/00109_image.jpg    (112, 224)
video_imades/class_0/video_00002/00110_image.jpg    (112, 224)
video_imades/class_0/video_00002/00111_image.jpg    (112, 224)
video_imades/class_0/video_00002/00112_image.jpg    (112, 224)
video_imades/class_0/video_00002/00113_image.jpg    (112, 224)
video_imades/class_0/video_00002/00114_image.jpg    (112, 224)
video_imades/class_0/video_00002/00115_image.jpg    (112, 224)
video_imades/class_0/video_00002/00116_image.jpg    (112, 224)
video_imades/class_0/video_00002/00117_image.jpg    (112, 224)
video_imades/class_0/video_00002/00118_image.jpg    (112, 224)
video_imades/class_0/video_00002/00119_image.jpg    (112, 224)
video_imades/class_0/video_00002/00120_image.jpg    (112, 224)
video_imades/class_0/video_00002/00121_image.jpg    (11

video_imades/class_0/video_00003/00007_image.jpg    (112, 224)
video_imades/class_0/video_00003/00008_image.jpg    (112, 224)
video_imades/class_0/video_00003/00009_image.jpg    (112, 224)
video_imades/class_0/video_00003/00010_image.jpg    (112, 224)
video_imades/class_0/video_00003/00011_image.jpg    (112, 224)
video_imades/class_0/video_00003/00012_image.jpg    (112, 224)
video_imades/class_0/video_00003/00013_image.jpg    (112, 224)
video_imades/class_0/video_00003/00014_image.jpg    (112, 224)
video_imades/class_0/video_00003/00015_image.jpg    (112, 224)
video_imades/class_0/video_00003/00016_image.jpg    (112, 224)
video_imades/class_0/video_00003/00017_image.jpg    (112, 224)
video_imades/class_0/video_00003/00018_image.jpg    (112, 224)
video_imades/class_0/video_00003/00019_image.jpg    (112, 224)
video_imades/class_0/video_00003/00020_image.jpg    (112, 224)
video_imades/class_0/video_00003/00021_image.jpg    (112, 224)
video_imades/class_0/video_00003/00022_image.jpg    (11

video_imades/class_0/video_00003/00138_image.jpg    (112, 224)
video_imades/class_0/video_00003/00139_image.jpg    (112, 224)
video_imades/class_0/video_00003/00140_image.jpg    (112, 224)
video_imades/class_0/video_00003/00141_image.jpg    (112, 224)
video_imades/class_0/video_00003/00142_image.jpg    (112, 224)
video_imades/class_0/video_00003/00143_image.jpg    (112, 224)
video_imades/class_0/video_00003/00144_image.jpg    (112, 224)
video_imades/class_0/video_00003/00145_image.jpg    (112, 224)
video_imades/class_0/video_00003/00146_image.jpg    (112, 224)
video_imades/class_0/video_00003/00147_image.jpg    (112, 224)
video_imades/class_0/video_00003/00148_image.jpg    (112, 224)
video_imades/class_0/video_00003/00149_image.jpg    (112, 224)
video_imades/class_0/video_00003/00150_image.jpg    (112, 224)
video_imades/class_0/video_00003/00151_image.jpg    (112, 224)
video_imades/class_0/video_00003/00152_image.jpg    (112, 224)
video_imades/class_0/video_00003/00153_image.jpg    (11

...

video_imades/class_1/video_00074/00029_image.jpg    (112, 224)
video_imades/class_1/video_00074/00030_image.jpg    (112, 224)
video_imades/class_1/video_00074/00031_image.jpg    (112, 224)
video_imades/class_1/video_00074/00032_image.jpg    (112, 224)
video_imades/class_1/video_00074/00033_image.jpg    (112, 224)
video_imades/class_1/video_00074/00034_image.jpg    (112, 224)
video_imades/class_1/video_00074/00035_image.jpg    (112, 224)
video_imades/class_1/video_00074/00036_image.jpg    (112, 224)
video_imades/class_1/video_00074/00037_image.jpg    (112, 224)
video_imades/class_1/video_00074/00038_image.jpg    (112, 224)
video_imades/class_1/video_00074/00039_image.jpg    (112, 224)
video_imades/class_1/video_00074/00040_image.jpg    (112, 224)
video_imades/class_1/video_00074/00041_image.jpg    (112, 224)
video_imades/class_1/video_00074/00042_image.jpg    (112, 224)
video_imades/class_1/video_00074/00043_image.jpg    (112, 224)
video_imades/class_1/video_00074/00044_image.jpg    (11

video_imades/class_1/video_00074/00161_image.jpg    (112, 224)
video_imades/class_1/video_00074/00162_image.jpg    (112, 224)
video_imades/class_1/video_00074/00163_image.jpg    (112, 224)
video_imades/class_1/video_00074/00164_image.jpg    (112, 224)
video_imades/class_1/video_00074/00165_image.jpg    (112, 224)
video_imades/class_1/video_00075/00001_image.jpg    (112, 224)
video_imades/class_1/video_00075/00002_image.jpg    (112, 224)
video_imades/class_1/video_00075/00003_image.jpg    (112, 224)
video_imades/class_1/video_00075/00004_image.jpg    (112, 224)
video_imades/class_1/video_00075/00005_image.jpg    (112, 224)
video_imades/class_1/video_00075/00006_image.jpg    (112, 224)
video_imades/class_1/video_00075/00007_image.jpg    (112, 224)
video_imades/class_1/video_00075/00008_image.jpg    (112, 224)
video_imades/class_1/video_00075/00009_image.jpg    (112, 224)
video_imades/class_1/video_00075/00010_image.jpg    (112, 224)
video_imades/class_1/video_00075/00011_image.jpg    (11

video_imades/class_1/video_00075/00129_image.jpg    (112, 224)
video_imades/class_1/video_00075/00130_image.jpg    (112, 224)
video_imades/class_1/video_00075/00131_image.jpg    (112, 224)
video_imades/class_1/video_00075/00132_image.jpg    (112, 224)
video_imades/class_1/video_00076/00001_image.jpg    (112, 224)
video_imades/class_1/video_00076/00002_image.jpg    (112, 224)
video_imades/class_1/video_00076/00003_image.jpg    (112, 224)
video_imades/class_1/video_00076/00004_image.jpg    (112, 224)
video_imades/class_1/video_00076/00005_image.jpg    (112, 224)
video_imades/class_1/video_00076/00006_image.jpg    (112, 224)
video_imades/class_1/video_00076/00007_image.jpg    (112, 224)
video_imades/class_1/video_00076/00008_image.jpg    (112, 224)
video_imades/class_1/video_00076/00009_image.jpg    (112, 224)
video_imades/class_1/video_00076/00010_image.jpg    (112, 224)
video_imades/class_1/video_00076/00011_image.jpg    (112, 224)
video_imades/class_1/video_00076/00012_image.jpg    (11

video_imades/class_1/video_00076/00129_image.jpg    (112, 224)
video_imades/class_1/video_00076/00130_image.jpg    (112, 224)
video_imades/class_1/video_00076/00131_image.jpg    (112, 224)
video_imades/class_1/video_00076/00132_image.jpg    (112, 224)
video_imades/class_1/video_00077/00001_image.jpg    (112, 224)
video_imades/class_1/video_00077/00002_image.jpg    (112, 224)
video_imades/class_1/video_00077/00003_image.jpg    (112, 224)
video_imades/class_1/video_00077/00004_image.jpg    (112, 224)
video_imades/class_1/video_00077/00005_image.jpg    (112, 224)
video_imades/class_1/video_00077/00006_image.jpg    (112, 224)
video_imades/class_1/video_00077/00007_image.jpg    (112, 224)
video_imades/class_1/video_00077/00008_image.jpg    (112, 224)
video_imades/class_1/video_00077/00009_image.jpg    (112, 224)
video_imades/class_1/video_00077/00010_image.jpg    (112, 224)
video_imades/class_1/video_00077/00011_image.jpg    (112, 224)
video_imades/class_1/video_00077/00012_image.jpg    (11

video_imades/class_1/video_00077/00129_image.jpg    (112, 224)
video_imades/class_1/video_00077/00130_image.jpg    (112, 224)
video_imades/class_1/video_00077/00131_image.jpg    (112, 224)
video_imades/class_1/video_00077/00132_image.jpg    (112, 224)
video_imades/class_1/video_00078/00001_image.jpg    (112, 224)
video_imades/class_1/video_00078/00002_image.jpg    (112, 224)
video_imades/class_1/video_00078/00003_image.jpg    (112, 224)
video_imades/class_1/video_00078/00004_image.jpg    (112, 224)
video_imades/class_1/video_00078/00005_image.jpg    (112, 224)
video_imades/class_1/video_00078/00006_image.jpg    (112, 224)
video_imades/class_1/video_00078/00007_image.jpg    (112, 224)
video_imades/class_1/video_00078/00008_image.jpg    (112, 224)
video_imades/class_1/video_00078/00009_image.jpg    (112, 224)
video_imades/class_1/video_00078/00010_image.jpg    (112, 224)
video_imades/class_1/video_00078/00011_image.jpg    (112, 224)
video_imades/class_1/video_00078/00012_image.jpg    (11

In [23]:
import shutil
from sklearn.model_selection import train_test_split

class Images_Preprocessing(Video_Preprocessing):
    """
    serve to preprocessing sets of images to create a train and a test folders of image's folders
    may use as video preprocesing to convert the video file  to the image's folders by period , 
    size  and max_length
    if video seperated by a black frame each video will be write in its own folder 
    video_file is the name of video file
    in this case needs the video_files or video_foldes
    """

    def __init__(self, folders=None, **kwards):
        super(Image_preprocesing, self).__init__()
        self.folders = folders # the folder contains image's folder gor each class
        #redefine parameters of the class
        for k in kwards.keys():
            self.__dict__[k] = kwards[k]
        

    
    
    def copy_files_to_folder(self, list_files, folder):
        """
        copy the list of files in the folder
        """
        # check the existing of the folder if not make
        if not os.path.exists(os.path.join(os.getcwd(),folder)):
            os.makedirs(os.path.join(os.getcwd(), folder))
        
        lable = os.path.split(folder)[-1]
        for f in list_files:
            _file = os.path.basename(f)
            _dir = os.path.split(os.path.dirname(f))[-1]
            if _dir != lable:
                if not os.path.exists(os.path.join(os.getcwd(), folder, _dir)):
                    os.makedirs(os.path.join(os.getcwd(), folder, _dir))
                    
                shutil.copyfile(f , os.path.join(os.getcwd(), folder, _dir, _file))
            else:
                shutil.copyfile(f , os.path.join(os.getcwd(), folder, _file))
            
    
    def create_train_test_folders(self, video_folder=None, test_size=0.15):
        """
        seperate images on train and test foldes 
        list images - list of image' paths 
        test_size is the size of the test folder 
        """
        if self.folders is None:
            try:
                self.different_video_folders_to_image_folders(video_folder)
            except TypeError:
                print 'need the video folder'
        else:
            folder = os.path.dirname(self.folders[0])
        
        list_images = sorted(self.get_list_of_images_path(folder))
        _list_images = map(lambda x: x.split('/'), list_images)
        
        mask = np.array(map(lambda x: not ('train' in x ) or ('test' in x), _list_images))
        list_images = np.array(list_images)[mask]
        y = np.vectorize(lambda x: x.split('/')[-3])(list_images)
    
        images_train, images_test, y_train, y_test = train_test_split(list_images, y, stratify=y, 
                                                                      test_size=test_size)
    
        for fd, img, y in zip(['train', 'test'], (images_train, images_test), (y_train, y_test)):
            for lable in np.unique(y):
                self.copy_files_to_folder(img[y == lable], os.path.join(folder, fd, lable))
                print 'write the folder', os.path.join(folder, fd, lable)
                
    
    
    def get_list_of_images_path(self, folder):
        """
        create a list of all images' paths  for all subfolders in the folder
        """
        fd_list = []
    
        if not self.all_dir(folder):
        # add to the list only files not dir
            return sorted(map(lambda x: os.path.join(folder, x), os.listdir(folder)))
    
        for fd in os.listdir(folder):
            path = os.path.join(os.getcwd(), folder, fd)
            fd_list += self.get_list_of_images_path(path)

        return fd_list 

    def all_dir(self, folder):
        """
        serve to check all filenames in folder is dir
        """
        flag = True
        if os.path.isdir(folder):
            for f in os.listdir(folder):
                flag = flag & os.path.isdir(os.path.join(folder,f))
            
        else:
            flag = False
        
        return flag    


In [38]:
cc_folders = ['video_images/class_0', 'video_images/class_1']

In [39]:
cc_folders = np.vectorize(lambda x: os.path.join(os.getcwd(), x))(cc_folders)

In [40]:
cc = Images_Preprocessing(size=(224, 112), folders=cc_folders)

In [41]:
cc.size

(224, 112)

In [42]:
#cc.different_video_folders_to_image_folders('video')

In [43]:
cc.create_train_test_folders()

write the folder /home/oleg/EnkeSystem/video_images/train/class_0
write the folder /home/oleg/EnkeSystem/video_images/train/class_1
write the folder /home/oleg/EnkeSystem/video_images/test/class_0
write the folder /home/oleg/EnkeSystem/video_images/test/class_1


In [51]:
os.listdir('/home/oleg/EnkeSystem/video_images/class_0/video_00001')

['00135_image.jpg',
 '00118_image.jpg',
 '00044_image.jpg',
 '00114_image.jpg',
 '00085_image.jpg',
 '00053_image.jpg',
 '00064_image.jpg',
 '00089_image.jpg',
 '00113_image.jpg',
 '00050_image.jpg',
 '00060_image.jpg',
 '00055_image.jpg',
 '00087_image.jpg',
 '00054_image.jpg',
 '00100_image.jpg',
 '00083_image.jpg',
 '00082_image.jpg',
 '00110_image.jpg',
 '00006_image.jpg',
 '00049_image.jpg',
 '00102_image.jpg',
 '00068_image.jpg',
 '00035_image.jpg',
 '00124_image.jpg',
 '00125_image.jpg',
 '00065_image.jpg',
 '00150_image.jpg',
 '00148_image.jpg',
 '00007_image.jpg',
 '00122_image.jpg',
 '00132_image.jpg',
 '00131_image.jpg',
 '00041_image.jpg',
 '00031_image.jpg',
 '00033_image.jpg',
 '00011_image.jpg',
 '00016_image.jpg',
 '00080_image.jpg',
 '00072_image.jpg',
 '00153_image.jpg',
 '00061_image.jpg',
 '00073_image.jpg',
 '00101_image.jpg',
 '00003_image.jpg',
 '00092_image.jpg',
 '00091_image.jpg',
 '00036_image.jpg',
 '00077_image.jpg',
 '00062_image.jpg',
 '00120_image.jpg',


In [27]:
def test_class_Images_to_sequence():
    
    folders = ['video_images/class_0', 'video_images/class_1']
    folders = np.vectorize(lambda x: os.path.join(os.getcwd(), x))(folders)
    cc = Images_to_sequence(folders=folders)
    print cc.f_lists

In [69]:
#libraries
import tensorflow as tf
import tflearn
tf.__version__

/home/oleg/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.10.0'

In [70]:
cnn_model = tf.keras.models.load_model('cnn_model.h5')
cnn_Model = tf.keras.Model(inputs=cnn_model.input,
                   outputs=cnn_model.get_layer(index=-3).output)

print cnn_Model.output_shape

(None, 32)


In [97]:
folders = ['video_images/class_0', 'video_images/class_1']
folders = np.vectorize(lambda x: os.path.join(os.getcwd(), x))(folders)
cc = Images_to_sequence(folders=folders)
#cc.create_dict_images_lists()
cc.store_images_features(cnn_Model)


save X (799, 20, 32) in file /home/oleg/EnkeSystem/video_images/X_class_0
save X (798, 20, 32) in file /home/oleg/EnkeSystem/video_images/X_class_1


In [96]:
class Images_to_sequence(Video_Preprocessing):
    """
    to convert image's files to image's sequence  and feature extracting
    folders - folder contains the folders of images
    max_sequence - the maximum length of sequence
    period_sequence - period of images add to the sequnce
    """
    def __init__(self, 
                 folders=None, # list of image's folders
                 max_sequence=20, # max length of sequence
                 period_sequence=5, # period of images to add to the sequence
                **kwards): 
        super(Video_Preprocessing, self).__init__()
        self.max_sequence = max_sequence
        self.period_sequence = period_sequence
        self.folders = folders
        #redefine parameters of the class
        for k in kwards.keys():
            self.__dict__[k] = kwards[k]
    
    def get_folders(self):
        """
        check the folders are in the same folder
        """
        if not self.folders is None:
            
            if len(set(map(os.path.dirname, self.folders))) !=1:
                raise OSError('the folders of classes are not in the same folder')
            if len(map(os.path.split, self.folders)) == 1:
                _dif = self.folders[0]
                _dir_list = map(lambda x: os.path.join(_dir, x), os.listdir(_dir))
                mask = map(os.path.isdir, _dir_list)
                _dir_list = np.array(_dir_list)[mask]
                _dir_list = _dir_list[(_dir_list != 'train') & (_dir_list != 'test')]
                self.folders = map(lambda x: os.path.join(_dir, x), _dir_list)
                return
            else:
                
            
            
            
    def create_dict_images_lists(self, folders=None):
        """
        create the dictionary of each kyes as a list of image's path
        """
        if not folders is None:
            self.folders = folders
        elif self.folders is None:
            print 'need the folder with images'
            pass
        f_lists = {}
        for f in self.folders:
            f_lists[f] = self.get_list_sequence_images(f) # make the dictionary of images' lists
            
        self.f_lists = f_lists
        
            

    def get_list_sequence_images(self, path):
        """
        serve to make the 2D tensor from the list of images
        with max length and period
        return 2D tensor (number of images' sequences, max length)
        """
        
        images_list = np.vectorize(lambda x: os.path.join(os.getcwd(), path, x))(np.sort(os.listdir(path)))
        files_list = np.array([])
        for _folder in images_list:
            if len(files_list) == 0:
                files_list = self.create_list_of_images(_folder)
            files_list = self.create_list_of_images(_folder, f_lists=files_list)
        
        return files_list.reshape(len(files_list)//self.max_sequence, self.max_sequence)
     
            
    def create_list_of_images(self, folder, f_lists=np.array([])):
        """
        serve to make the list of video images as a sequence with the period 
        max_length  - the length of images' sequence
        return list of images as sequence in  a list of paths
        """
        f_list = sorted(os.listdir(folder))
        f_list = np.vectorize(lambda x: os.path.join(os.getcwd(), folder, x))(np.array(f_list)) # the full names of files
    
        ind_mask = (np.arange(len(f_list)) % self.period_sequence == 0) # mask for images with the period
    
        # if the number of images in the folder less then max length 
        # reduce the period or fill the sequence the same images
        period = self.period_sequence
        while (np.sum(ind_mask) < self.max_sequence) & (period > 1):
            period -=1
            ind_mask = np.arange(len(f_list)) % period == 0
            if (period ==1) & (np.sum(ind_mask) < self.max_sequence):
                f_list = np.sort(np.random.choice(f_list, size=self.max_sequence))
                return np.append(f_lists, f_list)
        
        # if the number of images in the folder  more then max length
        # images from folder are taken to create the image's sequence with the max length by shift the mask 
        for j in range(self.period_sequence):
            if j != 0:
                ind_mask[-1] = False
                ind_mask = np.roll(ind_mask, 1) # shift the period' mask to one step
            f_temp = f_list[ind_mask]  # images that match the period
        
            # separate the f_temp to image' sequence with the max length
            for i in range(len(f_temp) - self.max_sequence):
                _f_list = f_temp[i*self.max_sequence :(i+1)*self.max_sequence]
                if len(_f_list) == 0:
                    break
                if len(_f_list) < self.max_sequence:
                    _f_list = np.sort(np.random.choice(_f_list, size=self.max_sequence))
                    break
                f_lists = np.append(f_lists, _f_list)
        
        return f_lists
    
    def store_images_features(self, cnn_Model):
        """
        store the image's features in .ny file
        cnn_Model - cnn model to extract   features from images
        """
        self.create_dict_images_lists()
        
        for k in self.f_lists.keys():
            _file = os.path.split(k)[0] + '/X_' + os.path.split(k)[1]
            X = self.create_tensor_of_features(k, cnn_Model)
            np.save(_file, X)
            print 'save X', X.shape, 'in file', _file, '.npy'
    
        
    def create_tensor_of_features(self, key, model):
        """
        from the 2D tensor of images make 3D the data's features tensor using  the model to extract image's features
        image_list - 2D tensor of images' names that were united to make images' sequences
        model - the model to extract the image' features
        return the 3D tensor (number of images, length of images' sequence, number of image' features)    
        """
        image_list = self.f_lists[key]
        N = image_list.shape[0]
        l = image_list.shape[1]
        n, = model.output_shape[1:]
    
    
        X = np.zeros((N, l, n))
    
        for i in range(N):
            x = self.create_tensor_from_list_of_image_sequence(image_list[i])
            #print x.shape
            features = model.predict_on_batch(x[:, :, :, np.newaxis])
        
            #features = tf.nn.batch_normalization(features, )
            X[i] = features.reshape(features.shape[0], features.shape[1])
        
        return X
    
        
            
    def create_tensor_from_list_of_image_sequence(self, image_list):
        """
        get the tensor from lists of images
        """
        size = self.get_image_size()
    
        # chech How much chanal does the image have?
        if (len(size) == 2) or (size[2] ==1):
            w, h, c = size[0], size[1], 1
            X = np.full((len(image_list), w , h ), 0, dtype=np.int8)
        elif len(size) == 3 :
            w, h, c = size[0], size[1], 3
            X = np.full((len(image_list), w , h, c), 0, dtype=np.int8)

        for i,img in enumerate(image_list):
            image = tf.keras.preprocessing.image.load_img(img, target_size=size)
            if c == 1:
                X[i] = tf.keras.preprocessing.image.img_to_array(image)[:,:,0]
            elif c ==3:
                X[i] = tf.keras.preprocessing.image.img_to_array(image)
         
        return X

    def get_image_size(self):
        """
        return the size of images in the folder
        """
        try:
            path = os.path.join(os.getcwd(), self.folders[0])
        
            while os.path.isdir(path):
                filename = os.listdir(path)[0]
                path = os.path.join(path, filename)
            
        except OSError as e:
            print str(e)
            return

        return imageio.imread(path).shape


In [ ]:
class Video_prediction(Video_Preprocessing):
    """
    to convert a video file to image's features format and make the prediction useing the ML model
    """
    def __init__(self, cnn_Model, **kwards):
        super(Image_preprocesing, self).__init__()
        self.cnn_Model = cnn_Model
        #redefine parameters of the class
        for k in kwards.keys():
            self.__dict__[k] = kwards[k]
        
    def create_tensor_of_features(self, image_list, model):
        """
        from the 2D tensor of images make 3D the data's features tensor using  the model to extract image's features
        image_list - 2D tensor of images' names that were united to make images' sequences
        model - the model to extract the image' features
        return the 3D tensor (number of images, length of images' sequence, number of image' features)    
        """
        N = image_list.shape[0]
        l = image_list.shape[1]
        n,  = model.output_shape[1:]
    
    
        X = np.zeros((N, l, n))
    
        for i in range(N):
            x = create_tensor_from_list_of_image_sequence(image_list[i])
            #print x.shape
            features = model.predict_on_batch(x[:, :, :, np.newaxis])
        
            #features = tf.nn.batch_normalization(features, )
            X[i] = features.reshape(features.shape[0], features.shape[1])
        
        #return X
        pass
    
    def store_images_features(self):
        """
        store the image's features in .ny file
        """
        
        for k in self.f_lists.keys():
            _file = 'x_' + k
            X = create_tensor_of_features(self.f_lists[k], self.cnn_Model)
            np.save(_file, X)
            print 'save X', X.shape, 'in file', _file
            
    def create_tensor_from_list_of_image_sequence(self):
    """
    get the tensor from lists of images
    """
    size = self.get_image_size()
    
    # chech How much chanal does the image have?
    if (len(size) == 2) or (size[2] ==1):
        w, h, c = size[0], size[1], 1
        X = np.full((len(self.image_list), w , h ), 0, dtype=np.int8)
    elif len(size) == 3 :
        w, h, c = size[0], size[1], 3
        X = np.full((len(self.image_list), w , h, c), 0, dtype=np.int8)

    for i,img in enumerate(self.image_list):
        
        image = tf.keras.preprocessing.image.load_img(img, target_size=size)
        if c == 1:
            X[i] = tf.keras.preprocessing.image.img_to_array(image)[:,:,0]
        elif c ==3:
            X[i] = tf.keras.preprocessing.image.img_to_array(image)
         
    return X

    def get_image_size(self):
        """
        return the size of images in the folder
        """
        try:
            path = os.path.join(os.getcwd(), self.folder)
        
            while os.path.isdir(path):
                filename = os.listdir(path)[0]
                path = os.path.join(path, filename)
            
        except OSError as e:
            print str(e)
            return

        return imageio.imread(path).shape
